# Pinecone openai demo
https://docs.pinecone.io/integrations/openai

in progress....

In [1]:
# !pip install -qU \
#     pinecone-client==3.0.2 \
#     openai==1.10.0 \
#     datasets==2.16.1


In [2]:
from openai import OpenAI
from getpass import getpass
import os 

if os.environ.get('OAI_KEY'):
    oai_key = os.environ['OAI_KEY']
else:
    oai_key = getpass("Enter an OPENAI key: ")
    
client = OpenAI(api_key=oai_key)

Enter an OPENAI key:  ········


In [14]:
def create_embeddings(text, model):
    res = client.embeddings.create(input=text, model=model)
    return res.data


In [15]:
model = "text-embedding-3-small"

test_res = create_embeddings(['simple embedding test'], model)

In [16]:
emb0 = test_res[0].embedding  #['embedding'][:10]

In [17]:
len(emb0)
print(f"First 10 embeddings of first text: {emb0[:10]}")

First 10 embeddings of first text: [0.01645466312766075, -0.03789207711815834, 0.011913697235286236, -0.045482080429792404, 0.009226778522133827, -0.01229030080139637, 0.025493139401078224, 0.005953227635473013, 0.005105870310217142, 0.014166074804961681]


In [18]:
import time
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="87c571a7-ecea-42ee-a5e3-6a1c4db88bad")

spec = ServerlessSpec(cloud="aws", region="us-east-1")

index_name = 'pinecone-openai-demo'

# check if index already exists (it shouldn't if this is your first run)
if index_name not in pc.list_indexes().names():
    print(f"index({index_name}) does not exist, create index")
    pc.create_index(
        index_name,
        dimension=len(embeds[0]),  # dimensionality of text-embed-3-small
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
else:
    print(f"index({index_name}) DOES exist, using existing...")
    
# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()


index(pinecone-openai-demo) DOES exist, using existing...


{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6}

In [19]:
lines = ['1a The boy zipped up his fly after putting on his pants',
         '1b The lad zipped his pants and washed his hands.',
         
         '2a The catcher pointed to the fly ball and the pitcher found it',
         '2b The pitcher caught the popup for the 3rd out in the inning.',
         
         '3a I opened the tent fly flaps and enjoyed the fresh air.',
         '3b to get fresh air I opened the tent flaps.'
        ]

In [20]:
emb_list = create_embeddings(lines, model)

In [21]:
# emb_list

In [22]:
pc_docs = []
for idx, emb in enumerate(emb_list):
    line = lines[idx]
    emb = emb_list[idx].embedding
    # print(f"Emb: {emb}")
    doc = {"id":str(idx), 'values':emb, 'metadata':{'number':str(idx), 'content':line}}
    pc_docs.append(doc)

In [23]:
foo = index.upsert(vectors=pc_docs)
foo

{'upserted_count': 6}

In [13]:
for line in lines:
    emb = 

SyntaxError: invalid syntax (373920689.py, line 2)